In [ ]:
# 📄 Customer Churn Dataset Generator

Generates synthetic customer data for a hosting/domain service business with churn labels driven by configurable behavioral patterns.

## Features
- Pattern-based churn logic (not random)
- Realistic data distributions
- Configurable parameters
- Fast generation (<2s for 100 records)


In [ ]:
# Install dependencies if needed
# !pip install pandas numpy faker

import pandas as pd
import numpy as np
from faker import Faker
import uuid
from datetime import datetime, timedelta
import warnings
import os
warnings.filterwarnings('ignore')

print("✅ Dependencies loaded successfully!")


In [ ]:
## 🔧 Configuration

Modify these parameters to customize your dataset:


In [ ]:
# Configuration
N_CUSTOMERS = 1000  # Number of customer records
RANDOM_SEED = 42   # Set to None for random results, or integer for reproducible results
OUTPUT_FILE = 'synthetic_churn_dataset.csv'
REFERENCE_DATE = '2024-12-01'  # Reference date for data generation

# Set seeds for reproducibility
if RANDOM_SEED is not None:
    np.random.seed(RANDOM_SEED)
    Faker.seed(RANDOM_SEED)

fake = Faker()
reference_date = datetime.strptime(REFERENCE_DATE, "%Y-%m-%d")
print(f"📊 Configured to generate {N_CUSTOMERS} records with seed={RANDOM_SEED}")
print(f"📅 Reference date: {REFERENCE_DATE}")


In [ ]:
## 🎲 Data Generation Functions


In [ ]:
def generate_basic_info(n_customers):
    """Generate basic customer and service information."""
    # Generate service creation dates (spread over past 3 years)
    max_days_ago = 3 * 365
    days_ago = np.random.randint(1, max_days_ago, n_customers)
    service_created_dates = [
        (reference_date - timedelta(days=int(d))).date()
        for d in days_ago
    ]
    
    # Generate expiration dates (1-24 months after creation)
    expiration_dates = []
    deletion_dates = []
    
    for created_date in service_created_dates:
        # Term length (typically 1, 12, or 24 months)
        term_choices = [1, 3, 6, 12, 24]
        term = np.random.choice(term_choices, p=[0.1, 0.2, 0.2, 0.4, 0.1])
        expiration_date = created_date + timedelta(days=int(term * 30))
        expiration_dates.append(expiration_date)
        
        # Some services are deleted (10% chance)
        if np.random.random() < 0.1:
            deletion_date = created_date + timedelta(days=int(np.random.randint(1, term * 30)))
            deletion_dates.append(deletion_date)
        else:
            deletion_dates.append(None)
    
    return {
        'FIRST_OF_MONTH': [reference_date.replace(day=1).date()] * n_customers,
        'ACCOUNT_ID': [10000 + i for i in range(n_customers)],
        'PERSON_ORG_ID': [20000 + i for i in range(n_customers)],
        'PP_SERVICE_CREATED_DATE': service_created_dates,
        'PP_SERVICE_DELETION_DATE': deletion_dates,
        'PP_EXPIRATION_DATE': expiration_dates,
        'PP_PREMIUM_FLAG': np.random.choice(['Y', 'N'], n_customers, p=[0.15, 0.85]),
        'PP_BUNDLE_FLAG': np.random.choice(['Y', 'N'], n_customers, p=[0.3, 0.7]),
        'STATE': np.random.choice(['CA', 'TX', 'NY', 'FL', 'IL', 'PA', 'OH'], n_customers)
    }

def generate_financial_metrics(bundle_flags, premium_flags, n_customers):
    """Generate financial and product metrics."""
    # Base pricing varies by product type
    base_amounts = np.random.lognormal(mean=3.2, sigma=0.8, size=n_customers)
    base_amounts = np.clip(base_amounts, 10, 500)
    
    # Premium domains cost more
    premium_multiplier = np.where(np.array(premium_flags) == 'Y', 
                                 np.random.uniform(2, 10, n_customers), 1)
    base_amounts *= premium_multiplier
    
    # Bundle discounts
    bundle_discount = np.where(np.array(bundle_flags) == 'Y',
                              np.random.uniform(0.1, 0.3, n_customers), 0)
    
    pp_net_amt = base_amounts
    pp_discount_amt = base_amounts * bundle_discount
    
    # Term lengths
    term_choices = [1, 3, 6, 12, 24]
    term_probs = [0.1, 0.2, 0.2, 0.4, 0.1]
    terms = np.random.choice(term_choices, n_customers, p=term_probs)
    
    # Renewal counts based on service age
    renewal_counts = np.random.poisson(lam=2, size=n_customers)
    renewal_counts = np.clip(renewal_counts, 0, 10)
    
    return {
        'PP_DISCOUNT_AMT': np.round(pp_discount_amt, 2),
        'PP_NET_AMT': np.round(pp_net_amt, 2),
        'TERM_IN_MONTHS': terms,
        'RENEWAL_COUNT': renewal_counts,
    }

def generate_product_counts_and_amounts(n_customers):
    """Generate product counts and corresponding amounts."""
    # All products
    all_product_cnt = np.random.poisson(lam=3, size=n_customers)
    all_product_cnt = np.clip(all_product_cnt, 1, 15)
    all_product_net_amt = all_product_cnt * np.random.lognormal(3, 0.5, n_customers)
    
    # Individual product categories (each is subset of all products)
    product_categories = [
        'ECOMMERC', 'ADV_HOSTING', 'HOSTING', 'DIY_WEBSIT', 'PRO_SERVICES',
        'HOSTING_ADD_ONS', 'EMAIL_PRODUCTIVITY', 'SECURITY', 'PREMIUM_DOMAIN', 
        'DOMAIN_VAS', 'DOMAIN'
    ]
    
    product_data = {}
    
    for category in product_categories:
        # Each category gets 0 to some fraction of all products
        max_for_category = np.maximum(1, all_product_cnt // 3)
        counts = np.random.poisson(lam=0.8, size=n_customers)
        counts = np.minimum(counts, max_for_category)
        
        # Amounts correlate with counts
        amounts = counts * np.random.lognormal(2.5, 0.8, n_customers)
        
        product_data[f'{category}_PRODUCT_CNT'] = counts
        product_data[f'{category}_NET_AMT'] = np.round(amounts, 2)
    
    product_data['ALL_PRODUCT_CNT'] = all_product_cnt
    product_data['ALL_PRODUCT_NET_AMT'] = np.round(all_product_net_amt, 2)
    
    # Add DOMAIN_NET_AMT as sum of domain-related amounts
    product_data['DOMAIN_NET_AMT'] = np.round(
        product_data['DOMAIN_PRODUCT_CNT'] * np.random.lognormal(2.5, 0.8, n_customers), 2
    )
    
    # Generate flags
    product_data['HAS_ECOMMERCE'] = np.where(product_data['ECOMMERC_PRODUCT_CNT'] > 0, 'Y', 'N')
    product_data['HAS_WORDPRESS'] = np.random.choice(['Y', 'N'], n_customers, p=[0.4, 0.6])
    
    return product_data

def generate_nps_and_support_metrics(n_customers):
    """Generate NPS scores and support contact metrics."""
    # NPS scores
    nps_promoter = np.random.poisson(lam=0.3, size=n_customers)
    nps_detractor = np.random.poisson(lam=0.2, size=n_customers)
    
    # Support contacts by category
    support_categories = [
        'TOTAL', 'WORDPRESS', 'DOMAIN', 'EMAIL', 'CPANEL', 'ACCOUNT', 
        'BILLING', 'RETENTION', 'SALES', 'SSL', 'TOS'
    ]
    
    support_data = {}
    
    # Generate total contacts first
    total_contacts = np.random.poisson(lam=2, size=n_customers)
    total_contacts = np.clip(total_contacts, 0, 20)
    support_data['TOTAL_CONTACTS'] = total_contacts
    
    # Other contact types are subsets of total
    for category in support_categories[1:]:  # Skip TOTAL
        if category == 'BILLING':
            # Billing contacts are more critical indicator
            contacts = np.random.poisson(lam=0.5, size=n_customers)
        elif category == 'RETENTION':
            # Retention contacts are red flag
            contacts = np.random.poisson(lam=0.2, size=n_customers)
        else:
            # Regular support contacts
            contacts = np.random.poisson(lam=0.8, size=n_customers)
        
        # Ensure subset of total
        contacts = np.minimum(contacts, total_contacts)
        support_data[f'{category}_CONTACTS'] = contacts
    
    support_data['NPS_PROMOTER_COUNT'] = nps_promoter
    support_data['NPS_DETRACTOR_COUNT'] = nps_detractor
    
    return support_data

def generate_login_metrics(n_customers):
    """Generate login activity metrics."""
    # Total login attempts
    total_logins = np.random.poisson(lam=10, size=n_customers)
    total_logins = np.clip(total_logins, 0, 100)
    
    # Success rate varies (most customers have good success rate)
    success_rates = np.random.beta(a=8, b=2, size=n_customers)
    successful_logins = np.round(total_logins * success_rates).astype(int)
    
    return {
        'SUCCESS_LOGIN': successful_logins,
        'TOTAL_LOGIN': total_logins
    }

print("✅ Data generation functions defined!")


In [ ]:
## 🎯 Churn Logic

This implements the rule-based scoring system from the PRD:


In [ ]:
def calculate_churn_score(df):
    """Calculate churn score based on rule-based logic from PRD."""
    scores = np.zeros(len(df))
    
    # Calculate service age
    today = reference_date
    service_ages = (today - pd.to_datetime(df['PP_SERVICE_CREATED_DATE'])).dt.days
    
    # Calculate days to expiration
    days_to_expiration = (pd.to_datetime(df['PP_EXPIRATION_DATE']) - today).dt.days
    
    # Apply scoring rules from PRD
    conditions = [
        ('Service Age < 90 days', service_ages < 90, 2),
        ('Service Expiring Soon (< 30 days)', days_to_expiration < 30, 3),
        ('Low Product Diversity (< 2)', df['ALL_PRODUCT_CNT'] < 2, 1),
        ('Low Spend (< $50)', df['ALL_PRODUCT_NET_AMT'] < 50, 2),
        ('No Renewals', df['RENEWAL_COUNT'] == 0, 2),
        ('High Support Contact (> 5)', df['TOTAL_CONTACTS'] > 5, 1),
        ('Billing Issues (> 2)', df['BILLING_CONTACTS'] > 2, 2),
        ('Retention Calls (> 0)', df['RETENTION_CONTACTS'] > 0, 1),
        ('Low Login Activity (0 success OR < 5 total)', 
         (df['SUCCESS_LOGIN'] == 0) | (df['TOTAL_LOGIN'] < 5), 2),
        ('NPS Detractor (> 0)', df['NPS_DETRACTOR_COUNT'] > 0, 1),
        ('Premium Domain Issues', 
         (df['PP_PREMIUM_FLAG'] == 'Y') & (df['PREMIUM_DOMAIN_PRODUCT_CNT'] == 0), 1),
        ('Short Term Contract (≤ 3 months)', df['TERM_IN_MONTHS'] <= 3, 1),
        ('High Discount Dependency (> 50%)', 
         df['PP_DISCOUNT_AMT'] / np.maximum(df['PP_NET_AMT'], 1) > 0.5, 1),
        ('No Core Products', 
         (df['HOSTING_PRODUCT_CNT'] == 0) & (df['DOMAIN_PRODUCT_CNT'] == 0), 2),
        ('Technical Support Issues (> 3)', 
         df['CPANEL_CONTACTS'] + df['EMAIL_CONTACTS'] > 3, 1)
    ]
    
    # Sum up the scores and show breakdown
    print("🎯 Churn Scoring Rules Applied:")
    for rule_name, condition, weight in conditions:
        rule_score = condition.sum()
        scores += (condition.astype(int) * weight)
        print(f"   {rule_name}: {rule_score} customers affected (weight: {weight})")
    
    return scores

def assign_churn_status(df, scores):
    """Assign churn status based on scores and override rules."""
    # Base assignment: score >= 8 means inactive
    base_status = ['inactive' if score >= 8 else 'active' for score in scores]
    
    # Apply override rules
    final_status = base_status.copy()
    
    # Automatic Active Status (Override)
    active_override = (
        (df['NPS_PROMOTER_COUNT'] > 0) & (df['ALL_PRODUCT_NET_AMT'] > 100)
    ) | (
        (df['RENEWAL_COUNT'] > 3) & (df['ALL_PRODUCT_CNT'] > 5)
    )
    
    # Automatic Inactive Status (Override)
    inactive_override = (
        df['PP_SERVICE_DELETION_DATE'].notna()
    ) | (
        df['BILLING_CONTACTS'] > 5
    )
    
    # Apply overrides
    active_overrides_applied = 0
    inactive_overrides_applied = 0
    
    for i in range(len(final_status)):
        if active_override.iloc[i]:
            final_status[i] = 'active'
            active_overrides_applied += 1
        elif inactive_override.iloc[i]:
            final_status[i] = 'inactive'
            inactive_overrides_applied += 1
    
    # Add 10% noise (random flips)
    noise_indices = np.random.choice(
        len(final_status), 
        size=int(0.1 * len(final_status)), 
        replace=False
    )
    
    for idx in noise_indices:
        final_status[idx] = 'inactive' if final_status[idx] == 'active' else 'active'
    
    print(f"\n📊 Churn Assignment Summary:")
    print(f"   Base rule assignment: {base_status.count('inactive')} inactive / {len(base_status)} total")
    print(f"   Active overrides applied: {active_overrides_applied}")
    print(f"   Inactive overrides applied: {inactive_overrides_applied}")
    print(f"   After 10% noise: {final_status.count('inactive')} inactive / {len(final_status)} total")
    
    return final_status

print("✅ Churn logic functions defined!")


In [ ]:
## 🚀 Generate Dataset

Run this cell to generate your synthetic churn dataset:


In [ ]:
%%time

print(f"🎲 Generating {N_CUSTOMERS} customer records...\n")

# Generate all data components
basic_info = generate_basic_info(N_CUSTOMERS)
financial_metrics = generate_financial_metrics(
    basic_info['PP_BUNDLE_FLAG'], 
    basic_info['PP_PREMIUM_FLAG'],
    N_CUSTOMERS
)
product_data = generate_product_counts_and_amounts(N_CUSTOMERS)
support_data = generate_nps_and_support_metrics(N_CUSTOMERS)
login_data = generate_login_metrics(N_CUSTOMERS)

# Combine into DataFrame
data = {**basic_info, **financial_metrics, **product_data, **support_data, **login_data}
df = pd.DataFrame(data)

# Calculate churn scores (no longer assigning status)
churn_scores = calculate_churn_score(df)

# Define column order to match PRD schema
column_order = [
    'FIRST_OF_MONTH', 'ACCOUNT_ID', 'PERSON_ORG_ID',
    'PP_SERVICE_CREATED_DATE', 'PP_SERVICE_DELETION_DATE', 'PP_EXPIRATION_DATE',
    'PP_PREMIUM_FLAG', 'PP_BUNDLE_FLAG', 'PP_DISCOUNT_AMT', 'PP_NET_AMT',
    'TERM_IN_MONTHS', 'RENEWAL_COUNT', 'ALL_PRODUCT_CNT', 'ALL_PRODUCT_NET_AMT',
    'ECOMMERC_PRODUCT_CNT', 'ECOMMERC_NET_AMT',
    'ADV_HOSTING_PRODUCT_CNT', 'ADV_HOSTING_NET_AMT',
    'HOSTING_PRODUCT_CNT', 'HOSTING_NET_AMT',
    'DIY_WEBSIT_PRODUCT_CNT', 'DIY_WEBSIT_NET_AMT',
    'PRO_SERVICES_PRODUCT_CNT', 'PRO_SERVICES_NET_AMT',
    'HOSTING_ADD_ONS_PRODUCT_CNT', 'HOSTING_ADD_ONS_NET_AMT',
    'EMAIL_PRODUCTIVITY_PRODUCT_CNT', 'EMAIL_PRODUCTIVITY_NET_AMT',
    'SECURITY_PRODUCT_CNT', 'SECURITY_NET_AMT',
    'PREMIUM_DOMAIN_PRODUCT_CNT', 'PREMIUM_DOMAIN_NET_AMT',
    'DOMAIN_VAS_PRODUCT_CNT', 'DOMAIN_VAS_NET_AMT',
    'DOMAIN_PRODUCT_CNT', 'DOMAIN_NET_AMT', 'STATE', 'HAS_ECOMMERCE', 'HAS_WORDPRESS',
    'NPS_PROMOTER_COUNT', 'NPS_DETRACTOR_COUNT',
    'TOTAL_CONTACTS', 'WORDPRESS_CONTACTS', 'DOMAIN_CONTACTS', 'EMAIL_CONTACTS',
    'CPANEL_CONTACTS', 'ACCOUNT_CONTACTS', 'BILLING_CONTACTS', 'RETENTION_CONTACTS',
    'SALES_CONTACTS', 'SSL_CONTACTS', 'TOS_CONTACTS',
    'SUCCESS_LOGIN', 'TOTAL_LOGIN'
]

df_final = df[column_order]

# Save to CSV
df_final.to_csv(OUTPUT_FILE, index=False)

print(f"\n✅ Dataset generated and saved to: {OUTPUT_FILE}")
print(f"📊 Shape: {df_final.shape}")
print(f"📋 Total columns: {len(df_final.columns)}")
# Churn status column removed - no longer showing churn rate

# Preview first 5 rows (key columns only)
preview_cols = [
    'ACCOUNT_ID', 'PP_SERVICE_CREATED_DATE', 'ALL_PRODUCT_CNT', 
    'ALL_PRODUCT_NET_AMT', 'TOTAL_CONTACTS'
]
print(f"\n📋 Preview (first 5 rows, key columns):")
display(df_final[preview_cols].head())

print(f"\n📋 Full column list ({len(df_final.columns)} total):")
for i, col in enumerate(df_final.columns, 1):
    print(f"{i:2d}. {col}")
    if i % 5 == 0:  # Add line break every 5 columns for readability
        print()
